This is the REGRESSION MODEL for Microsoft Azure submission.

In [22]:
#Imports 
# Azure Machine Learning SDK core
from azureml.core import Workspace
from azureml.core.model import Model

# Scikit-learn and others
from sklearn import datasets; iris = datasets.load_iris()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Most of these aren't used, but are here in case test code needs to be run.

In [23]:
# Connect with workspace
ws = Workspace.from_config(path="config.json")

Note that this next step, registering the model, is commented - we don't need to register the model more than once depending on the cases.

In [24]:
# Registering Model
# model = Model.register(ws, model_name="random-forest-regression", model_path="reg_model.pkl")

Next - the actual code to insert into the model every time we're making predictions.

In [25]:
# 'Consume' Code

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
    "data": [[0,1,3,1,1,2,2,1,3.0,1,2,93,91]]  #JSON ISSUE !!!
}

body = str.encode(json.dumps(data))

url = 'https://msa-phase2-azure-xnuor.australiaeast.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = '6uHYccyHVeC7pZ0NFhVuIAZhQQyKcKgP'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'random-forest-regression-2-final' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


The request failed with status code: 424
server: azureml-frontdoor
date: Wed, 09 Aug 2023 10:28:26 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/0.8.4
x-ms-request-id: 561e0cb4-0262-4b43-aef1-e255f45bfd1b
x-request-id: 561e0cb4-0262-4b43-aef1-e255f45bfd1b
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: random-forest-regression-2-final
connection: close


{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}


-- Remove if functional --
I've set up within Azure (which has been seamless), but these errors, which correlate with JSON data types are always present. I'm working on a fix, but may not be able to find one before the final time to submit. I've made multiple deployments and score.py iterations.

However, the model works, as per the training and eval section.